In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

from flask import jsonify
from flask import request
from flask_api import FlaskAPI
from sqlalchemy import create_engine
import json

In [21]:
#Data Scrapping
base_URL = "https://www.freshfields.com"
URL = "https://www.freshfields.com/en-gb/contacts/find-a-lawyer/?Name=&t=&Service=97&Role=&Location=&Office=&Industry=&Page="
columns = ["URL","Name","Role","Fax","Mobile","Telephone","Address","Country","Email","TimeStamp"]

def get_base_content(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id = "main-content")
    return results

def details_links():
    detail_links = []
    result = get_base_content(URL)
    navs = result.find("div",class_="row fal-flex").find_previous("div").find_all(alt="pageCount")
    for nav in navs:
        results = get_base_content(base_URL+nav['href'])
        contents = results.find("div", class_="row fal-flex")
        for i in contents.find_all('a'):
            detail_links.append(base_URL+i['href'])
    return detail_links

def user_details(URL):
    _user = []
    results = get_base_content(URL)
    name_role = results.find("a",class_="hero-link").find_previous("h1")
    _user.append(URL) #url
    _user.append(name_role.text.strip().split(", ")[0]) #name
    _user.append(name_role.text.strip().split(", ")[1]) #role
    
    adr = results.find("address")
    ads = []
    
    for i in adr.text.strip().split("\n"):
        if(len(i)>1):
            ads.append(i.strip())
            
    spl = " ".join(ads).split("F +")
    _user.append("+"+spl.pop().strip()) #fax
    
    spl = spl[0].split("M +")
    if(len(spl) == 2):
        _user.append("+"+spl.pop().strip()) #phone
    else:
        _user.append("null")
        
    spl = spl[0].split("T +")
    _user.append("+"+spl.pop().strip()) #telephone
    
    _user.append(spl.pop().strip()) #address
    
    _user.append(results.find("address").find_previous('h3').text) #country
    _user.append(results.find("address").find_next('a').text) #mail
    _user.append(datetime.datetime.now()) #TimeStamp
    return dict(zip(columns,_user))

def get_dataFrame():
    f_l = []
    x = details_links()
    for i in x:
        f_l.append(user_details(i))
    df = pd.DataFrame(f_l)
    return df

In [16]:
data = get_dataFrame() #execute to get the dataframe
data

,URL,Name,Role,Fax,Mobile,Telephone,Address,Country,Email,TimeStamp
0,https://www.freshfields.com/en-gb/contacts/fin...,Fares Al-Hejailan,Partner,+966 1 479 1717,null,+966 11 479 2200,Freshfields Bruckhaus Deringer LLP in associat...,Riyadh,fares.al-hejailan@freshfields.com,2021-11-22 19:40:40.560181
1,https://www.freshfields.com/en-gb/contacts/fin...,David Almroth,Partner,+1 646 521 5670,null,+1 212 277 4070,"601 Lexington Avenue, 31st Floor New York, NY ...",New York,david.almroth@freshfields.com,2021-11-22 19:40:41.269376
2,https://www.freshfields.com/en-gb/contacts/fin...,Aled Batey,Partner,+44 20 7108 2481,null,+44 20 7785 2481,100 Bishopsgate London EC2P 2SR,London,aled.batey@freshfields.com,2021-11-22 19:40:42.034162
3,https://www.freshfields.com/en-gb/contacts/fin...,Simone Bono,Partner,+44 20 7832 7001,null,+44 20 7832 7269,100 Bishopsgate London EC2P 2SR,London,simone.bono@freshfields.com,2021-11-22 19:40:42.820365
4,https://www.freshfields.com/en-gb/contacts/fin...,Christopher Davis,Partner,+44 20 7832 7001,null,+44 20 7936 4000,100 Bishopsgate London EC2P 2SR,London,christopher.davis@freshfields.com,2021-11-22 19:40:43.538502
5,https://www.freshfields.com/en-gb/contacts/fin...,Thijs P. Flokstra,Partner/Notaris,+31 20 485 7001,+31 6 53 17 33 28,+31 20 4857620,Strawinskylaan 10 1077 XZ Amsterdam,Amsterdam,thijs.flokstra@freshfields.com,2021-11-22 19:40:44.315878
6,https://www.freshfields.com/en-gb/contacts/fin...,Daniel French,Partner,+852 2810 6192,null,+852 2913 2600,"55th Floor, One Island East Taikoo Place, Quar...",Hong Kong,daniel.french@freshfields.com,2021-11-22 19:40:44.979740
7,https://www.freshfields.com/en-gb/contacts/fin...,Iñaki Gabilondo,Partner,+34 91 308 4636,null,+34 91 700 3700,"Torre Europa Paseo de la Castellana, 95 Madrid...",Madrid,inaki.gabilondo@freshfields.com,2021-11-22 19:40:45.843189
8,https://www.freshfields.com/en-gb/contacts/fin...,Andrew Hagan,Partner,+44 20 7108 7434,null,+44 20 7832 7434,100 Bishopsgate London EC2P 2SR,London,andrew.hagan@freshfields.com,2021-11-22 19:40:46.444029
9,https://www.freshfields.com/en-gb/contacts/fin...,Dr. Mario Hüther,Partner,+49 69 23 26 64,null,+49 69 27 30 80,Bockenheimer Anlage 44 60322 Frankfurt am Main,Frankfurt am Main,mario.huether@freshfields.com,2021-11-22 19:40:47.176002


In [22]:
#API
app = FlaskAPI(__name__)

e = create_engine('sqlite:///requests_flask_sqlite.db')


@app.route('/create_table', methods=['POST']) #need to pass the table name Eg: /create_table?tbl=tbl_name
def create_table():
    try:
        tbl = request.args.get('tbl')
        conn = e.connect()
        conn.execute('CREATE TABLE IF NOT EXISTS {} (SNO INTEGER PRIMARY KEY AUTOINCREMENT,URL TEXT,NAME TEXT,ROLE TEXT,ADDRESS TEXT,COUNTRY TEXT,MOBILE TEXT,TELEPHONE TEXT,FAX TEXT,EMAIL TEXT,TIMESTAMP TEXT);'.format(tbl))
        text = {"Message " : "Table created successfully"}
        return text
    except Exception as error:
        text = {"Message " : "Table creation failed : {}".format(error)}
        return text
    finally:
        conn.close()

@app.route('/tbl_view_all', methods=['GET']) #Eg: /tbl_view_all?tbl=tbl_name
def tbl_view_all():
    try:
        tbl = request.args.get('tbl')
        conn=e.connect()
        result = conn.execute("select * from {}".format(tbl))
        return json.dumps([dict(r) for r in result])
    except Exception as error:
        text = {"Message " : "Table not found : {}".format(error)}
        return text
    finally:
        conn.close()

@app.route('/tbl_view_one', methods=['GET']) #Eg: /tbl_view_one?tbl=tbl_name&url=https://www.freshfields.com/en-gb/contacts/find-a-lawyer/f/french-daniel/
def tbl_view_one():
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        url = all_args.get('url')
        result = conn.execute("select * from {} where URL = '{}';".format(tbl,url))  
        return json.dumps([dict(r) for r in result])
    
    except Exception as error:
        text = {"Message " : "Record not found : {}".format(error)}
        return text
    finally:
        conn.close()
    
@app.route('/tbl_insert_one', methods=['POST']) #Eg: /tbl_insert_one?tbl=tbl_name&url=https://www.freshfields.com/en-gb/contacts/find-a-lawyer/f/french-daniel/
def tbl_insert_one():    
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        url = all_args.get('url')
        result = user_details(url)
        conn.execute("INSERT INTO {} (URL,NAME,ROLE,ADDRESS,COUNTRY,MOBILE,TELEPHONE,FAX,EMAIL,TIMESTAMP) VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');".format(tbl,result.get("URL"),result.get("Name"),result.get("Role"),result.get("Address"),result.get("Country"),result.get("Mobile"),result.get("Telephone"),result.get("Fax"),result.get("Email"),result.get('TimeStamp')))
        text = {"Message " : "Record inserted successfully"}
        return text
    
    except Exception as error:
        text = {"Message " : "Insertion failed : {}".format(error)}
        return text
    finally:
        conn.close()
    
@app.route('/freshfields_insert', methods=['POST']) #Eg: /freshfields_insert?tbl=tbl_name
def freshfields_insert():    
    try:
        conn = e.connect()
        all_args = request.args.to_dict()
        tbl = all_args.get('tbl')
        data.to_sql(tbl, conn, if_exists='append',index=False)  #insert data using dataframe (created with scrapped data)
        text = {"Message " : "Records inserted successfully"}
        return text
    
    except Exception as error:
        text = {"Message " : "Insertion failed : {}".format(error)}
        return text
    finally:
        conn.close()
    
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2021 19:46:17] "POST /create_table?tbl=freshfields HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 19:46:20] "GET /tbl_view_all?tbl=freshfields HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 19:46:23] "POST /freshfields_insert?tbl=freshfields HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 19:46:25] "GET /tbl_view_all?tbl=freshfields HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 19:46:28] "GET /tbl_view_one?tbl=freshfields&url=https://www.freshfields.com/en-gb/contacts/find-a-lawyer/a/almroth-david/ HTTP/1.1" 200 -
